In [37]:
import networkx as nx
import numpy as np
import random

def is_independent_set(graph, subset):
    for u in subset:
        for v in subset:
            if u != v and graph.has_edge(u, v):
                return False
    return True

def is_dominating_set(graph, subset):
    dominating_nodes = set(subset)
    for node in subset:
        dominating_nodes.update(graph.neighbors(node))
    return len(dominating_nodes) == len(graph.nodes)

def fitness(solution, graph):
    if is_independent_set(graph, solution) and is_dominating_set(graph, solution):
        return 1 / len(solution)  # Valid solution: inversely proportional to size
    return float('-inf')  # Invalid solution

def aco_minimum_independent_dominating_set(graph, num_ants, num_iterations, alpha, beta, evaporation_rate):
    node_list = list(graph.nodes())  # Get a list of the graph's nodes
    num_nodes = len(node_list)
    pheromone_matrix = np.ones(num_nodes)  # Initialize pheromone levels for each node

    best_solution = None
    best_fitness = float('-inf')

    for iteration in range(num_iterations):
        solutions = []

        for _ in range(num_ants):
            # Generate a solution (set of nodes)
            solution = set()
            available_nodes = set(node_list)  # Set of nodes still available for selection
            
            while available_nodes:  # Continue until no nodes are left
                # Choose a node based on pheromone levels
                probabilities = np.array([pheromone_matrix[node_list.index(node)] for node in available_nodes])
                probabilities /= probabilities.sum()  # Normalize probabilities
                
                selected_node = np.random.choice(list(available_nodes), p=probabilities)
                solution.add(selected_node)
                available_nodes.remove(selected_node)  # Remove selected node
                
                # Check if the solution is still valid
                if not is_independent_set(graph, solution):  # Check validity
                    solution.remove(selected_node)  # Remove if it makes it invalid
                    break
            
            fitness_value = fitness(solution, graph)
            solutions.append((solution, fitness_value))

            # Update the best solution found so far
            if fitness_value > best_fitness:  # Maximizing fitness
                best_fitness = fitness_value
                best_solution = solution

        # Update pheromones
        pheromone_matrix *= (1 - evaporation_rate)  # Evaporate pheromones
        for solution, fitness_value in solutions:
            if fitness_value > 0:  # Only update for valid solutions
                for node in solution:
                    pheromone_matrix[node_list.index(node)] += (1 / fitness_value)  # Increase pheromone

    return best_solution, best_fitness

# Example Usage
if __name__ == "__main__":
    graph = nx.read_gml("tests/test_100_0.3.in")

    best_set, best_value = aco_minimum_independent_dominating_set(graph, num_ants=10, num_iterations=1000, alpha=1, beta=1, evaporation_rate=0.1)
    print("Best Independent Dominating Set:", best_set)
    print("Fitness (1 / length of the set):", best_value)


Best Independent Dominating Set: None
Fitness (1 / length of the set): -inf
